# Load SNP orders

In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "SNP"

# Set the root
from from_root import from_root # type: ignore
ROOT = from_root()

import pandas as pd # type: ignore
from loguru import logger # type: ignore

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util # type: ignore

util.startLoop()

logger.add(sink=ROOT / "log" / "snp.log", mode="w")

## Helper functions

In [3]:
def yes_or_no(question: str, default="n") -> bool:
    """Asks a yes or no question with a default answer.

    Args:
      question: The question to ask.
      default: The default answer if the user presses Enter.

    Returns:
      True if the user answered yes, False otherwise.
    """

    while True:
        answer = input(question + " (y/n): ").lower().strip()
        if not answer:
            return default == "y"
        if answer in ("y", "yes"):
            return True
        elif answer in ("n", "no"):
            return False
        else:
            print("Please answer yes or no.")

# Get orders to be placed

In [ ]:
# Check age of pickles

from src.snp import get_pickle, how_many_days_old


nakeds_path = ROOT / 'data' / 'snp_nakeds.pkl'
txt = f'snp_nakeds.pkl is {how_many_days_old(nakeds_path): 0.2f} days old. Want to load??'
ans = yes_or_no(txt)

if ans:
    df_opts = get_pickle(nakeds_path)
    print('\n\n')

    cols = 'ib_symbol,strike,safe_strike,right,dte,bsPrice,price,xPrice,margin,rom'.split(',')
    print(df_opts[cols].head())
else:
    print('Bye!!!')

# Remove open orders from orders to plant

In [ ]:
from src.snp import get_ib, get_open_orders, quick_pf

# check open orders
with get_ib('SNP') as ib:
    dfo = get_open_orders(ib)
    dfp = quick_pf(ib)

if not dfo.empty:
    remove_opens = set(dfo.symbol.to_list())
else:
    remove_opens = set()

# make a list of symbols to be removed from df_opts
if not dfp.empty:
    remove_positions = set(dfp.symbol.to_list())
else:
    remove_positions = set()

remove_ib_syms = remove_opens | remove_positions

# get the target options to plant
dft = df_opts[~df_opts.ib_symbol.isin(remove_ib_syms)].reset_index(drop=True)

print(f'{len(dft)} target orders found\n\n')

dft.head()

# PREPARE THE ORDERS

In [75]:
from src.snp import load_config

config = load_config(MARKET)
MARGINPERORDER = config.get('MARGINPERORDER')

dfa = dft.groupby('ib_symbol').apply(lambda x: x.sort_values('rom', ascending=False), include_groups=False).reset_index(level='ib_symbol')

KeyError: 'Requested level (ib_symbol) does not match index name (None)'

In [74]:
dfa.groupby('ib_symbol').margin.transform('sum')

462    19285.0
164    19285.0
884    15669.6
387    15669.6
851    41804.6
        ...   
503    30529.0
476    30529.0
407    30529.0
436    30529.0
463    30529.0
Name: margin, Length: 1109, dtype: float64

In [ ]:
# Calculate n for each ib_symbol
dfa['n'] = MARGINPERORDER // dfa.groupby('ib_symbol')['margin'].transform('sum')

# Filter dfa to keep only the top n entries for each ib_symbol
dfa = dfa.groupby('ib_symbol').head(dfa['n']).reset_index(drop=True)



In [72]:
dfa

,ib_symbol,conId,expiry,strike,right,contract,price,undId,secType,undPrice,und_iv,und_hv,dte,iv,sdev,safe_strike,intrinsic,bsPrice,margin,comm,lot,xPrice,rom,n
462,AAPL,733773996,2024-10-25 10:00:00+00:00,232.5,P,"Option(conId=733773996, symbol='AAPL', lastTra...",2.66,265598,STK,232.15,0.24,0.23,4.75,0.24,6.39,224,224.0,2.64,9517.0,0.65,1,11.16,9.01,0.0
164,AAPL,735049342,2024-11-01 10:00:00+00:00,232.5,P,"Option(conId=735049342, symbol='AAPL', lastTra...",5.17,265598,STK,232.15,0.24,0.23,11.75,0.24,10.05,219,219.0,3.98,9768.0,0.65,1,18.67,5.94,0.0
884,ABBV,733773777,2024-10-25 10:00:00+00:00,187.5,P,"Option(conId=733773777, symbol='ABBV', lastTra...",1.76,118089500,STK,188.57,0.30,0.18,4.75,0.30,6.36,180,180.0,1.97,7718.8,0.65,1,10.54,10.50,0.0
387,ABBV,735049385,2024-11-01 10:00:00+00:00,187.5,P,"Option(conId=735049385, symbol='ABBV', lastTra...",4.08,118089500,STK,188.57,0.30,0.18,11.75,0.30,10.01,176,176.0,3.32,7950.8,0.65,1,16.65,6.51,0.0
851,ABNB,733172751,2024-10-25 10:00:00+00:00,136.0,P,"Option(conId=733172751, symbol='ABNB', lastTra...",2.14,459530964,STK,136.02,0.44,0.31,4.75,0.44,6.84,127,127.0,2.67,5654.8,0.65,1,11.69,15.88,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,XOM,728314455,2024-11-01 10:00:00+00:00,120.0,P,"Option(conId=728314455, symbol='XOM', lastTrad...",1.95,13977,STK,120.35,0.24,0.23,11.75,0.24,5.07,114,114.0,1.75,5009.0,0.65,1,8.30,5.15,0.0
476,XOM,731982738,2024-11-08 10:00:00+00:00,120.0,P,"Option(conId=731982738, symbol='XOM', lastTrad...",2.59,13977,STK,120.35,0.24,0.23,18.75,0.24,6.41,112,112.0,2.22,5073.0,0.65,1,10.94,4.20,0.0
407,XOM,722321206,2024-11-15 10:00:00+00:00,120.0,P,"Option(conId=722321206, symbol='XOM', lastTrad...",3.32,13977,STK,120.35,0.24,0.23,25.75,0.24,7.51,110,110.0,2.60,5146.0,0.65,1,13.67,3.77,0.0
436,XOM,733299456,2024-11-22 10:00:00+00:00,120.0,P,"Option(conId=733299456, symbol='XOM', lastTrad...",3.60,13977,STK,120.35,0.24,0.23,32.75,0.24,8.47,109,109.0,2.92,5174.0,0.65,1,14.95,3.22,0.0


In [ ]:

margin_per_symbol = dft.groupby('ib_symbol').margin.sum().div(MARGINPERORDER).astype(int)
dfa = dft.groupby('ib_symbol').apply(lambda x: x.sort_values('rom', ascending=False).head(margin_per_symbol[x.name]), include_groups=False).reset_index()

df= dfa.groupby('ib_symbol').apply(lambda x: x.margin.sum(), include_groups=False)

In [ ]:
margin_per_symbol[margin_per_symbol.index == 'BKNG']

In [ ]:
df.sort_values(ascending=False).astype('int')

In [28]:
from ib_async import LimitOrder

def make_ib_orders(df: pd.DataFrame) -> tuple:
    """Make (contract, order) tuples"""

    contracts = df.contract.to_list()
    orders = [
        LimitOrder(action="SELL", totalQuantity=abs(int(q)), lmtPrice=p)
        for q, p in zip(df.lot, df.xPrice)
    ]

    cos = tuple((c, o) for c, o in zip(contracts, orders))

    return cos

In [25]:
cos = make_ib_orders(dft)

# PLACE THE ORDERS